activate conda environment keras.yml

In [2]:
import keras, numpy as np, os, pandas as pd, time, random
import subprocess, re, sys, os
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers.core import Lambda
from keras import backend as K
from keras.models import Model
import tensorflow
from keras import metrics

In [3]:
dim = 3
seed = 374173
traindir = 'training/'

In [13]:
#Specify testset
out = 
KMERS = out+'gambiae_complex/kmers.npy'
SAMPLES = out+'gambiae_complex/samples.csv'

In [14]:
testkmer = np.load(KMERS)
testmeta = pd.read_csv(SAMPLES)
testnames = testmeta['SUPPLIER SAMPLE NAME'].values
testamplified = testmeta['n_mosquito_targets'].values

In [8]:
#load model
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                             mean=0., stddev=1.,seed=seed)
    return z_mean + K.exp(z_log_var) * epsilon

In [9]:
width = 128
latent_dim=dim
depth=6
batch_size =32

In [10]:
#encoder
input_seq = keras.Input(shape=(4**8,))
x=layers.Dense(width,activation="elu")(input_seq)
for i in range(depth-1):
    x=layers.Dense(width,activation="elu")(x)
z_mean=layers.Dense(latent_dim)(x)
z_log_var=layers.Dense(latent_dim)(x)
z = layers.Lambda(sampling,output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
encoder=Model(input_seq,[z_mean,z_log_var,z],name='encoder')

#decoder
decoder_input=layers.Input(shape=(latent_dim,),name='z_sampling')
x=layers.Dense(width,activation="linear")(decoder_input)
for i in range(depth-1):
    x=layers.Dense(width,activation="elu")(x)
output=layers.Dense(4**8,activation="softplus")(x) #can also use other activations that return strictly positive lambda parameter values
decoder=Model(decoder_input,output,name='decoder')

#end-to-end vae
output_seq = decoder(encoder(input_seq)[2])
vae = Model(input_seq, output_seq, name='vae')

In [11]:
#predict latent space coords for all samples from weights minimizing val loss
vae.load_weights(traindir+"_weights.hdf5")
pred=encoder.predict(testkmer,batch_size=batch_size) #returns [mean,sd,sample] for individual distributions in latent space
p=pd.DataFrame()

p['mean1']=pred[0][:,0]
p['mean2']=pred[0][:,1]
p['mean3']=pred[0][:,2]
p['sd1']=pred[1][:,0]
p['sd2']=pred[1][:,1]
p['sd3']=pred[1][:,2]
pred=p

pred['sample_id']=testnames
pred['n_mosquito_targets']=testamplified

pred.to_csv(out+"gambiae_complex/latent_coords.csv", index=False)

In [12]:
pred.shape

(192, 9)

In [13]:
pred

,mean1,mean2,mean3,sd1,sd2,sd3,sample_id,amplified,sample_set
0,-72.516068,-26.807144,1.379827,-35.862930,-33.388355,-17.564011,VBS49531-6367STDY10108697,61,1264-VO-CD-WATSENGA-VMF00164
1,-83.754318,-38.048141,5.834773,-38.709469,-38.133286,-19.958004,VBS49514-6367STDY10108678,61,1264-VO-CD-WATSENGA-VMF00164
2,-82.968567,-31.640045,1.573283,-39.427715,-36.838497,-20.185228,VBS49346-6367STDY10108521,61,1264-VO-CD-WATSENGA-VMF00164
3,-87.582085,-45.285194,9.853935,-39.755886,-41.142803,-20.831636,VBS49476-6367STDY10108647,61,1264-VO-CD-WATSENGA-VMF00164
4,-89.720230,-35.857395,2.700749,-42.039307,-39.692905,-21.706537,VBS49481-6367STDY10108652,62,1264-VO-CD-WATSENGA-VMF00164
...,...,...,...,...,...,...,...,...,...
187,-100.297028,-39.234825,1.940520,-46.175491,-43.477207,-24.440285,VBS49371-6367STDY10108545,62,1264-VO-CD-WATSENGA-VMF00164
188,-108.496323,-48.609203,6.302334,-48.480412,-47.752865,-25.808863,VBS49342-6367STDY10108517,62,1264-VO-CD-WATSENGA-VMF00164
189,-88.389000,-36.037556,3.107330,-41.347839,-39.131767,-21.484484,VBS49555-6367STDY10108721,62,1264-VO-CD-WATSENGA-VMF00164
190,-83.373764,-25.309166,-3.286617,-41.364384,-36.235832,-20.746767,VBS49470-6367STDY10108641,61,1264-VO-CD-WATSENGA-VMF00164
